# Figures for attribution presentations

## pdf shift plot

### Gaussian

In [210]:
library(shape)

x <- seq(24,35,0.1)

mean_f <- 30.562; sd_f <- 1.343
mean_cf <- mean_f - 1.2; sd_cf <- sd_f - 0.1

obs <- 32 #qnorm(0.8, mean_f, sd_f)
xx <- seq(obs,35,0.01)

png("pdf_gaussian_shiftscale.png", height = 480, width = 480); {

    # set plot parameters (scale text by 1.5x, set outer margins to remove whitespace)
    par(cex = 1.5, mar = c(4,2,1,1))
    
    # create an empty plot - will probably need to change height of y-axis via second value in ylim
    plot(0, type = "n", xaxs = "i", yaxs = "i", bty = "n", yaxt = "n", xlim = c(min(x) - 0.2, max(x)), ylim = c(0,0.425), xlab = "", ylab = "")
    ymax <- par("usr")[4]*0.925

    # axes & axis labels
    mtext(expression("Temperature ("*degree*"C)"), side = 1, line = 2.5, cex = 1.5)
    mtext("Likelihood", side = 2, cex = 1.5)
    Arrows(min(x),0,min(x),ymax, lwd = 3)
    Arrows(min(x),0,max(x),0, lwd = 3)

    # add lines showing counterfactual & factual densities
    lines(x, dnorm(x, mean = mean_cf, sd = sd_cf), type = "l", lwd = 3, col = "blue")
    lines(x, dnorm(x, mean = mean_f, sd = sd_f), type = "l", lwd = 3, col = "red3")
    abline(h = 0, lwd = 3)

    # add shaded polygons - currently non-overlapping 
    polygon(c(xx, rev(xx)), c(rep(0,length(xx)), rev(dnorm(xx, mean = mean_cf, sd = sd_cf))), col = adjustcolor("blue", alpha = 0.3), border = NA)
    polygon(c(xx, rev(xx)), c(dnorm(xx, mean = mean_cf, sd = sd_cf), rev(dnorm(xx, mean = mean_f, sd = sd_f))), col = adjustcolor("red", alpha = 0.3), border = NA)

    # line marking observed temperature
    lines(rep(obs,2), c(0,ymax), col = "dimgrey", lty = 2, lwd = 3)
    text(obs, ymax, "Observed", col = "dimgrey", pos = 3)
    

    # add labels (will need to play around with x & y coordinates to get best location)
    text(28.3,.225, "Cooler\nclimate", pos = 2, col = "blue")
    # text(26.5,.2, expression(cooler), pos = 2, col = "blue")

    text(30.6,.33, "Current\nclimate", col = "red3")

    text(33.3,.07, "P1", col = "red3")
    text(33,.02, "P0", col = "blue")

}; dev.off()

png 
  2

### GEV

In [226]:
library(shape)

x <- seq(25,36,0.1)

mean_f <- 30.562; sd_f <- 1.343; xi <- -0.2
mean_cf <- mean_f - 1.2; sd_cf <- sd_f - 0.1

obs <- 32
xx <- seq(obs,max(x),0.01)

png("pdf_gev_shiftscale.png", height = 480, width = 480); {

    # set plot parameters (scale text by 1.5x, set outer margins to remove whitespace)
    par(cex = 1.5, mar = c(4,2,1,1))
    
    # create an empty plot - will probably need to change height of y-axis via second value in ylim
    plot(0, type = "n", xaxs = "i", yaxs = "i", bty = "n", yaxt = "n", xlim = c(min(x) - 0.2, max(x)), ylim = c(0,0.465), xlab = "", ylab = "")
    ymax <- par("usr")[4]*0.925

    # axes & axis labels
    mtext(expression("Temperature ("*degree*"C)"), side = 1, line = 2.5, cex = 1.5)
    mtext("Likelihood", side = 2, cex = 1.5)
    Arrows(min(x),0,min(x),ymax, lwd = 3)
    Arrows(min(x),0,max(x),0, lwd = 3)

    # add lines showing counterfactual & factual densities
    lines(x, devd(x, loc = mean_cf, scale = sd_cf, shape = xi), type = "l", lwd = 3, col = "blue")
    lines(x, devd(x, loc = mean_f, scale = sd_f, shape = xi), type = "l", lwd = 3, col = "red3")
    abline(h = 0, lwd = 3)

    # add shaded polygons - currently non-overlapping 
    polygon(c(xx, rev(xx)), c(rep(0,length(xx)), rev(devd(xx, loc = mean_cf, scale = sd_cf, shape = xi))), col = adjustcolor("blue", alpha = 0.3), border = NA)
    polygon(c(xx, rev(xx)), c(devd(xx, loc = mean_cf, scale = sd_cf, shape = xi), rev(devd(xx, loc = mean_f, scale = sd_f, shape = xi))), col = adjustcolor("red", alpha = 0.3), border = NA)

    # line marking observed temperature
    lines(rep(obs,2), c(0,ymax), col = "dimgrey", lty = 2, lwd = 3)
    text(obs, ymax, "Observed", col = "dimgrey", pos = 3)
    

    # add labels (will need to play around with x & y coordinates to get best location)
    text(28.3,.225, "Cooler\nclimate", pos = 2, col = "blue")
    # text(26.5,.2, expression(cooler), pos = 2, col = "blue")

    text(30.6,.33, "Current\nclimate", col = "red3")

    text(33,.07, "P1", col = "red3")
    text(32.4,.02, "P0", col = "blue")

}; dev.off()

png 
  2

# CET time series plots

In [115]:
suppressMessages(library(rwwa))
library(shape)

df <- merge(read.csv("cet_tx1x.csv", col.names = c("year", "tmax")), read.csv("gmst.csv", col.names = c("year", "gmst")))
df$gmst <- df$gmst - df$gmst[df$year == 2022]
df <- df[df$year < 2025,]

mdl <- fit_ns("gev", "shift", df, "tmax","gmst", lower = F, ev_year = 2022)
y_pi <- ns_pars(mdl, fixed_cov = data.frame("gmst" = -1.3))$loc
y_2022 <- ns_pars(mdl, fixed_cov = data.frame("gmst" = 0))$loc

loess_fit <- fitted(loess(tmax ~ gmst, df))

erl_10 <- eff_return_level(mdl, 10)[order(df$gmst)]

In [116]:
png("cet-timeseries-1.png", height = 480, width = 480*2); {
    plot(df[,c("year", "tmax")], type = "S", lwd = 2, xlab = "", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series")
    points(df[df$year == 2022,c("year", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
}; dev.off()

png 
  2

In [117]:
png("cet-timeseries-2_tmax-vs-gmst.png", height = 480, width = 480*2); {
    plot_covtrend(mdl, "gmst", lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab =  "Maximum temperature (degC)",
        main = "Central England Temperature series", add_loess = F)
}; dev.off()

png 
  2

In [118]:
png("cet-timeseries-3_tmax-vs-gmst-with-loess.png", height = 480, width = 480*2); {
    plot_covtrend(mdl, "gmst", lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab =  "Maximum temperature (degC)",
        main = "Central England Temperature series", add_loess = T)
}; dev.off()

png 
  2

In [119]:
png("cet-timeseries-4_tmax-vs-gmst-factual-counterfactual.png", height = 480, width = 480*2); {
    plot_covtrend(mdl, "gmst", lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab =  "Maximum temperature (degC)",
        main = "Central England Temperature series", add_loess = T, ci_cov = data.frame(gmst = c(0, -1.3)))
}; dev.off()

png 
  2

In [120]:
png("cet-timeseries-5_returnlevelplots.png", height = 480, width = 480*2); {
    plot_returnlevels(mdl, cov_f =  data.frame(gmst = 0), cov_cf = data.frame(gmst = -1.3), 
                      xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab =  "Maximum temperature (degC)",
                      main = "Central England Temperature series", model_desc = F, ylim = c(20,40), legend_pos = "right")
}; dev.off()

png 
  2

In [121]:
png("cet-timeseries-6_density-plots.png", height = 360, width = 360); {

    # it can also extract the model parameters for a fixed value of the covariates
    pars_factual <- ns_pars(mdl, fixed_cov = data.frame(gmst = 0))
    pars_counterfactual <- ns_pars(mdl, fixed_cov = data.frame(gmst = -1.3))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # we can therefore plot the density of the temperatures as they would have been in today's climate, or in the counterfactual climate
    par(lwd = 2, mar = c(4,4,1,1))
    # plot(density(y_factual), col = "red3", ylim = c(0,0.175), main = "", xlab = "Maximum temperature (degC)")
    plot(x <- seq(20,42,0.1), devd(x, loc = pars_factual$loc, scale = pars_factual$scale, shape = pars_factual$shape),
         type = "l", col = "red3", lty = 1, xlab = "Maximum temperature (degC)", ylab = "Density", ylim = c(0,0.25))
    
    # lines(density(y_counterfactual), col = "blue2")
    lines(x, devd(x, loc = pars_counterfactual$loc, scale = pars_counterfactual$scale, shape = pars_counterfactual$shape),
         type = "l", col = "blue2", lty = 1)
    
    # add ticks to show temperatures that were actually observed
    rug(df$tmax, lwd = 3)

    legend("topright", c("2024 climate", "Preindustrial climate"), col = c("red3", "blue2"), lwd = 3)
    
}; dev.off()

png 
  2

In [122]:
boot_res <- boot_ci(mdl, cov_f = data.frame(gmst = 0), cov_cf = data.frame(gmst = -1.3))

In [124]:
round(boot_res, 3)

est     2.5%    97.5% 
mu0              29.555  28.559 30.698
sigma0            2.014   1.727  2.246
alpha_gmst        2.614   1.576  3.724
shape            -0.183  -0.327 -0.071
disp              0.068   0.059  0.076
event_magnitude  37.300  37.300 37.300
return_period   767.000 118.158    Inf
PR                  Inf   9.766    Inf
dI_abs            3.398   2.183  4.876
dI_rel           10.022   6.218 15.038

---

In [9]:
png("cet-timeseries-2.png", height = 480, width = 480*2); {
    plot(df[,c("year", "tmax")], type = "S", lwd = 2, xlab = "", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series")
    points(df[df$year == 2022,c("year", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$year, fitted(loess(tmax ~ year, df)), col = "forestgreen", lw = 2)
}; dev.off()

png 
  2

In [10]:
png("cet-timeseries-3_vs-gmst.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
}; dev.off()

png 
  2

In [ ]:
png("cet-timeseries-4_loess.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
}; dev.off()

In [12]:
png("cet-timeseries-4_vs-gmst-loess.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
}; dev.off()

png 
  2

In [13]:
png("cet-timeseries-5_linear-model.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
}; dev.off()

png 
  2

In [14]:
png("cet-timeseries-4_linear-model-only.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
}; dev.off()

png 
  2

In [15]:
png("cet-timeseries-6_gmst-change.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    Arrows(-1.2,y_pi,0,y_pi, col = "orange", lw = 2, arr.adj = 1, arr.type = "curved")
    text(-0.4, y_pi-1, "GMST has increased by 1.2C", col = "orange", font = 2)

    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
}; dev.off()

png 
  2

In [16]:
png("cet-timeseries-7_tmax-change.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    Arrows(-1.2,y_pi,0,y_pi, col = "orange", lw = 2, arr.adj = 1, arr.type = "curved")
    text(-0.4, y_pi-1, "GMST has increased by 1.2C", col = "orange", font = 2)
    Arrows(0,y_pi,0,y_2022, col = "orange", lw = 2, arr.adj = 1, arr.type = "curved")
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
    text(0.02, y_2022-2.5, "Average\ntmax\n increases\nby 3C", col = "orange", font = 2, adj = 0)

}; dev.off()

png 
  2

In [17]:
png("cet-timeseries-8_return-levels.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
    lines(df$gmst[order(df$gmst)], erl_10, col = "darkred", lw = 2, lty = 2)
    text(0.05, erl_10[length(erl_10)], "1-in-10-year\nevent", adj = 0, col = "darkred", font = 2)
}; dev.off()

png 
  2

In [18]:
png("cet-timeseries-9_event-probability.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
    lines(df$gmst[order(df$gmst)], erl_10, col = "darkred", lw = 2, lty = 2)
    text(0.05, erl_10[length(erl_10)], "1-in-10-year\nevent", adj = 0, col = "darkred", font = 2)
    text(0.035, mdl$ev-0.2, "1-in-800-year\nevent", adj = 0, col = "darkred", font = 2)
}; dev.off()

png 
  2

In [19]:
png("cet-timeseries-9_event-probability2.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
    lines(df$gmst[order(df$gmst)], erl_10, col = "darkred", lw = 2, lty = 2)
    text(0.05, erl_10[length(erl_10)], "1-in-10-year\nevent", adj = 0, col = "darkred", font = 2)
    text(0.035, mdl$ev-0.2, "1-in-800-year\nevent", adj = 0, col = "darkred", font = 2)
}; dev.off()

png 
  2

# Scotland rainfall time series plots
https://www.metoffice.gov.uk/hadobs/hadukp/data/download.html

In [78]:
suppressMessages(library(rwwa))
library(shape)

df <- merge(read.csv("scotland_rx1x.csv", col.names = c("year", "rx1x")), read.csv("gmst.csv", col.names = c("year", "gmst")))
df$gmst <- df$gmst - df$gmst[df$year == 2024]
df <- df[df$year < 2025,]

mdl <- fit_ns("gev", "fixeddisp", df, "rx1x", "gmst", lower = F, ev_year = 2020)
y_pi <- ns_pars(mdl, fixed_cov = data.frame("gmst" = -1.2))$loc
y_2022 <- ns_pars(mdl, fixed_cov = data.frame("gmst" = 0))$loc

loess_fit <- fitted(loess(rx1x ~ gmst, df))

erl_10 <- eff_return_level(mdl, 10)[order(df$gmst)]

In [80]:
png("scotpr-timeseries-2_rx1x-vs-gmst.png", height = 480, width = 480*2); {
    plot_covtrend(mdl, "gmst", lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Annual maximum daily rainfall (mm)",
        main = "Daily rainfall accumulated over eastern Scotland")
}; dev.off()

png 
  2

# Figures for NAS panel

# Quick pdf plot


In [92]:
library(shape)

mean_f <- 30.562; sd_f <- 1.343
mean_cf <- mean_f - 1.2; sd_cf <- sd_f - 0.1
obs <- 32 #qnorm(0.8, mean_f, sd_f)
x <- seq(24,35,0.1)

In [119]:
qpdf_norm <- function(mean_f, sd_f, mean_cf, sd_cf, obs, x, xlab = "") {

    if(missing(x)) {
        xlim <- range(mean_f + 4*sd_f*c(-1,1), mean_cf + 4*sd_cf*c(-1,1))
        x <- seq(xlim[1],xlim[2], 0.01)
    }
    xx <- seq(obs,max(x),diff(x)[1]/10)

    dns_f <- dnorm(x, mean = mean_f, sd = sd_f)
    dns_cf <- dnorm(x, mean = mean_cf, sd = sd_cf)

    # create an empty plot - will probably need to change height of y-axis via second value in ylim
    plot(0, type = "n", xaxs = "i", yaxs = "i", bty = "n", yaxt = "n", xlim = c(min(x) - 0.2, max(x)), ylim = c(0,max(dns_f, dns_cf) * 1.2), xlab = "", ylab = "")
    ymax <- par("usr")[4]*0.925

    mtext(xlab, side = 1, line = 2.5, cex = 1.5)
    mtext("Likelihood", side = 2, cex = 1.5)

    # add lines showing counterfactual & factual densities
    lines(x, dns_cf, type = "l", lwd = 3, col = "blue")
    lines(x, dns_f, type = "l", lwd = 3, col = "red3")
    abline(h = 0, lwd = 3)

    # add shaded polygons - currently non-overlapping 
    polygon(c(xx, rev(xx)), c(rep(0,length(xx)), rev(dnorm(xx, mean = mean_cf, sd = sd_cf))), col = adjustcolor("blue", alpha = 0.3), border = NA)
    polygon(c(xx, rev(xx)), c(dnorm(xx, mean = mean_cf, sd = sd_cf), rev(dnorm(xx, mean = mean_f, sd = sd_f))), col = adjustcolor("red", alpha = 0.3), border = NA)

    # line marking observed temperature
    lines(rep(obs,2), c(0,ymax), col = "dimgrey", lty = 2, lwd = 3)
    text(obs, ymax, "Observed", col = "dimgrey", pos = 3)

    Arrows(min(x),0,min(x),ymax, lwd = 3)
    Arrows(min(x),0,max(x),0, lwd = 3, arr.adj = 1)

}


png 
  2

## Madagascar heatwave

In [ ]:
suppressMessages(library(rwwa))
library(shape)

df <- merge(read.table("gmst.txt", col.names = c("year", "gmst")), read.table("madagascar-heat.txt", col.names = c("year", "tx")))
df$gmst <- df$gmst - df$gmst[df$year == 2023]

cov_2023 <- df[df$year == 2023,"gmst",drop=F]
cov_pi <- cov_2023 - 1.2

In [31]:
mdl <- fit_ns("norm", "shift", data = df[df$year >= 1950,], varnm = "tx", covnm = "gmst")

In [151]:
png("gmsttrend_madagascar.png", height = 360, width = 620); par(mar = c(4,4,1,1)); {
    plot_covtrend(mdl, "gmst", add_loess = F, ylab = "October mean temperature (ERA5)", ci_cov = rbind(cov_2023, cov_pi), xlab = "GMST (degrees cooler than 2023)")
}; dev.off()

png 
  2

In [126]:
png("pdf_madagascar.png", height = 480, width = 480); par(mar = c(4,2,1,1)); {
qpdf_norm(mean_f = ns_pars(mdl, fixed_cov = cov_2023)$loc, sd_f = ns_pars(mdl, fixed_cov = cov_2023)$scale,
          mean_cf = ns_pars(mdl, fixed_cov = cov_pi)$loc, sd_cf = ns_pars(mdl, fixed_cov = cov_pi)$scale,
          obs = mdl$ev, xlab = "October mean temperature (degC)")
    }; dev.off()

png 
  2

In [128]:
png("rlplot_madagascar.png", height = 480, width = 480); par(mar = c(4,4,1,1)); {
    plot_returnlevels(mdl, cov_f = cov_2023, cov_cf = cov_pi, legend_pos = "topleft", ylab = "October mean temperature (degC)")
}; dev.off()

png 
  2

## Horn of Africa rainfall

In [46]:
suppressMessages(library(rwwa))

df <- merge(merge(read.table("gmst.txt", col.names = c("year", "gmst")),
                  read.table("iod_son.txt", col.names = c("year", "iod"))), 
            read.table("hoa-ond.txt", col.names = c("year", "pr")))
df$gmst <- df$gmst - df$gmst[df$year == 2023]
df$pr <- df$pr * 30 # correct to mm/30day

cov_2023 <- df[df$year == 2023,c("gmst", "iod"),drop=F]
cov_pi <- cov_2023 - c(1.2, 0)

In [47]:
mdl <- fit_ns("norm", "fixeddisp", data = df[df$year >= 1950,], varnm = "pr", covnm = c("gmst", "iod"))

In [56]:
png("hoa_rx30day-vs-year.png", h = 360, w = 540); {

    plot_trend(mdl, ylim = c(0,400), ylab = "rx30day in the Horn of Africa (mm)", main = "rx30day ~ GMST + IOD_SON")
}; dev.off()

png 
  2

In [57]:
png("hoa_rx30day-vs-iod.png", h = 360, w = 540); {
    plot_covtrend(mdl, "iod", ylim = c(0,400), ylab = "rx30day in the Horn of Africa (mm)", main = "rx30day ~ GMST + IOD_SON")
    abline(v = 0, lty = 2)
}; dev.off()


png 
  2